## **Getting started**

**Business Objective:**

goal is to understand what factors make a car more or less expensive. As a result of your analysis, you should provide clear recommendations to your client—a used car dealership—as to what consumers value in a used car.

**Problem Statement:** Determine which features are most strongly associated with what makes a car more or less expensive.


**Proposed Process Resolution:**
This notebook analyzes the factors influencing coupon acceptance (Y=1) versus non-acceptance (Y=0):
Coupon Acceptance/Rejection Fetures Discovery/Analysis/Correlation. Including data clensing, correlation analysis, and visualizations that compares key features.


**Findings:**
  **miles** and certain time thresholds (e.g., **toCoupon_GEQ15min** and **fuel**) are strong predictors of coupon acceptance.
  **make mdoel, like **temperature**, are more related to coupon rejection.



In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import joblib
import os
import json
import warnings
import pickle
warnings.filterwarnings('ignore')

import os
# from google.colab import files

from scipy.stats import chi2_contingency



# Advanced Vehicle Price Prediction Model

## Overview
This project implements an advanced machine learning model for predicting used vehicle prices using LightGBM (Light Gradient Boosting Machine). The model incorporates sophisticated feature engineering, proper data preprocessing, and robust prediction pipelines.

## Model Architecture

### 1. Data Preprocessing Pipeline
The model uses a comprehensive preprocessing pipeline implemented in the `VehiclePricePredictor` class:

```python
class VehiclePricePredictor:
    def __init__(self):
        self.model = None
        self.scaler = None
        self.feature_info = None
        self.feature_names = None
        self.category_mappings = {}
```

#### Key Preprocessing Steps:
- **Data Cleaning** (`clean_data()`):
  - Handles missing values
  - Removes outliers (prices < $100 or > $100,000)
  - Fills missing odometer readings with mean values
  - Fills missing years with median values
  - Handles missing categorical values with 'unknown'

- **Feature Engineering** (`engineer_features()`):
  - Vehicle age calculation (2024 - year)
  - Miles per year calculation
  - Price per mile (training only)
  - Price per year (training only)
  - Age-mileage interaction features

- **Data Preprocessing** (`preprocess_data()`):
  - Categorical variable standardization
  - One-hot encoding
  - Feature scaling
  - Feature name preservation

### 2. Model Training
The model uses LightGBM with optimized parameters:

```python
params = {
    'objective': 'regression',
    'metric': 'mae',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1
}
```

#### Training Process:
1. Data loading and cleaning
2. Feature engineering
3. Train/validation split
4. Model training with early stopping
5. Model evaluation using MAE and R² metrics
6. Artifact saving (model, scaler, feature info)

### 3. Prediction Pipeline
The prediction process follows these steps:
1. Input data validation
2. Feature preprocessing
3. Feature engineering
4. Model prediction
5. Price output

## Key Features

### 1. Robust Feature Engineering
- **Temporal Features**:
  - Vehicle age
  - Miles per year
  - Price per year (training)
  - Price per mile (training)

- **Interaction Features**:
  - Age-mileage interaction
  - Manufacturer-model combinations

### 2. Categorical Variable Handling
- Standardized lowercase conversion
- One-hot encoding
- Category mapping preservation
- Unknown category handling

### 3. Model Persistence
The model saves three key artifacts:
1. `model.joblib`: The trained LightGBM model
2. `scaler.pkl`: The StandardScaler for feature normalization
3. `feature_info.json`: Feature names and category mappings

## Usage

### Training the Model
```bash
python src/improved_model.py
```

### Making Predictions
```bash
python src/predict.py
```

### Using the GUI
```bash
python src/gui.py
```

## Performance Metrics
The model is evaluated using:
- Mean Absolute Error (MAE)
- R² Score
- Validation set performance
- Cross-validation results

## Dependencies
- pandas
- numpy
- lightgbm
- scikit-learn
- joblib
- pickle

## Future Improvements
1. Hyperparameter optimization
2. Additional feature engineering
3. Ensemble methods
4. Real-time price updates
5. Market trend analysis

In [3]:
#file list
files = os.listdir('../data/')
print(files)

['vehicles.csv.zip', 'archive.zip', 'vehicles.csv']


In [ ]:
# upload data csv files, uncomment
# from google.colab import files
# uploaded = files.upload()

In [7]:
#load uploaded csv file
cardata = pd.read_csv('../data/vehicles.csv')
print(cardata.count())

id              426880
url             426880
region          426880
region_url      426880
price           426880
year            425675
manufacturer    409234
model           421603
condition       252776
cylinders       249202
fuel            423867
odometer        422480
title_status    418638
transmission    424324
VIN             265838
drive           296313
size            120519
type            334022
paint_color     296677
image_url       426812
description     426810
county               0
state           426880
lat             420331
long            420331
posting_date    426812
dtype: int64


In [8]:
# Explore data
cardata.head()
# cpn.describe()
# cpn.info()
# cpn.isna().sum()



,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN


In [9]:
class VehiclePricePredictor:
    def __init__(self):
        self.model = None
        self.scaler = None
        self.feature_info = None
        self.feature_names = None
        self.category_mappings = {}

    def clean_data(self, df, is_training=True):
        """Clean the dataset"""
        # Drop rows with missing prices during training
        if is_training:
            df = df.dropna(subset=['price'])
            # Filter out unreasonable prices
            df = df[(df['price'] >= 100) & (df['price'] <= 100000)]

        # Fill missing values
        df['odometer'] = df['odometer'].fillna(df['odometer'].mean() if is_training else 50000)
        df['year'] = df['year'].fillna(df['year'].median() if is_training else 2020)

        # Fill missing categorical values with 'unknown'
        categorical_cols = ['manufacturer', 'model', 'condition', 'fuel', 'title_status',
                           'transmission', 'drive', 'type', 'paint_color']
        for col in categorical_cols:
            if col in df.columns:
                df[col] = df[col].fillna('unknown')

        return df

    def engineer_features(self, df, is_training=True):
        """Engineer additional features"""
        # Convert categorical columns to lowercase
        for col in ['manufacturer', 'model', 'condition', 'fuel', 'title_status', 'transmission', 'drive', 'type', 'paint_color']:
            if col in df.columns:
                df[col] = df[col].str.lower()

        # Calculate age
        df['age'] = 2024 - df['year']

        # Calculate mileage per year
        df['miles_per_year'] = df['odometer'] / (df['age'] + 1)

        # Only calculate price-related features during training
        if is_training:
            df['price_per_mile'] = df['price'] / (df['odometer'] + 1)
            df['price_per_year'] = df['price'] / (df['age'] + 1)

        # Create interaction features
        df['age_mileage'] = df['age'] * df['odometer']

        return df

    def preprocess_data(self, df, is_training=True):
        """Preprocess the data for model training or prediction"""
        # Convert categorical columns to lowercase
        categorical_cols = ['manufacturer', 'model', 'condition', 'fuel', 'title_status',
                           'transmission', 'drive', 'type', 'paint_color']
        for col in categorical_cols:
            if col in df.columns:
                df[col] = df[col].str.lower()

        # Calculate interaction features
        df['age_mileage'] = df['age'] * df['odometer']

        # During training, save the feature names and category mappings
        if is_training:
            # Get all possible categories for each categorical column
            self.category_mappings = {}
            for col in categorical_cols:
                if col in df.columns:
                    self.category_mappings[col] = sorted(df[col].unique().tolist())

            # Create dummy variables
            dummies = pd.get_dummies(df[categorical_cols])

            # Combine with numeric features
            numeric_features = ['year', 'odometer', 'age', 'miles_per_year', 'age_mileage']
            X = pd.concat([df[numeric_features], dummies], axis=1)

            # Save feature names
            self.feature_names = list(X.columns)

            # Return features and target
            y = df['price']
            return X, y

        # During prediction, ensure we have all the same features as during training
        dummies = pd.get_dummies(df[categorical_cols])

        # Combine with numeric features
        numeric_features = ['year', 'odometer', 'age', 'miles_per_year', 'age_mileage']
        X = pd.concat([df[numeric_features], dummies], axis=1)

        # Add missing columns with zeros
        for col in self.feature_names:
            if col not in X.columns:
                X[col] = 0

        # Select only the columns that were present during training
        X = X[self.feature_names]

        return X

    def train(self, data_path, nrows=None):
        """Train the model with improved feature engineering"""
        print("Loading and preparing data...")
        df = pd.read_csv(data_path, nrows=nrows)

        print("Cleaning data...")
        df = self.clean_data(df)

        print("Engineering features...")
        df = self.engineer_features(df)

        print("Preprocessing data...")
        X, y = self.preprocess_data(df)

        # Save feature names and category mappings
        self.feature_names = list(X.columns)
        print(f"Number of features: {len(self.feature_names)}")

        # Save feature info immediately after preprocessing
        os.makedirs('models', exist_ok=True)

        # Save feature names
        feature_names_path = os.path.join('models', 'feature_names.txt')
        with open(feature_names_path, 'w') as f:
            for feature in self.feature_names:
                f.write(feature + '\n')
        print(f"Feature names saved to {feature_names_path}")

        # Save category mappings
        category_mappings_path = os.path.join('models', 'category_mappings.txt')
        with open(category_mappings_path, 'w') as f:
            for col, categories in self.category_mappings.items():
                f.write(f"{col}:{','.join(categories)}\n")
        print(f"Category mappings saved to {category_mappings_path}")

        # Split data
        print("Splitting data...")
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

        # Scale features
        print("Scaling features...")
        self.scaler = StandardScaler()
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_val_scaled = self.scaler.transform(X_val)

        # Train model
        print("Training model...")
        self.model = lgb.train(
            {
                'objective': 'regression',
                'metric': 'l1',
                'boosting_type': 'gbdt',
                'num_leaves': 31,
                'learning_rate': 0.05,
                'feature_fraction': 0.9,
                'bagging_fraction': 0.8,
                'bagging_freq': 5,
                'verbose': -1
            },
            lgb.Dataset(X_train_scaled, label=y_train),
            num_boost_round=1000,
            valid_sets=[lgb.Dataset(X_val_scaled, label=y_val)],
            callbacks=[
                lgb.early_stopping(stopping_rounds=50),
                lgb.log_evaluation(period=100)
            ]
        )

        # Save model and artifacts
        print("Saving model and artifacts...")
        self.save()

        # Make a test prediction
        test_vehicle = {
            'year': 2020,
            'manufacturer': 'tesla',
            'model': 'model 3',
            'condition': 'excellent',
            'odometer': 25000,
            'fuel': 'electric',
            'title_status': 'clean',
            'transmission': 'automatic',
            'drive': 'rwd',
            'type': 'sedan',
            'paint_color': 'white'
        }
        test_df = pd.DataFrame([test_vehicle])
        test_df = self.clean_data(test_df, is_training=False)
        test_df = self.engineer_features(test_df, is_training=False)
        test_X = self.preprocess_data(test_df, is_training=False)
        test_X_scaled = self.scaler.transform(test_X)
        test_pred = self.model.predict(test_X_scaled)[0]
        print(f"\nPredicted price for test vehicle: ${test_pred:,.2f}")

    def predict(self, vehicle):
        """Make a price prediction for a single vehicle"""
        # Create DataFrame from vehicle data
        df = pd.DataFrame([vehicle])

        # Clean and engineer features
        df = self.clean_data(df, is_training=False)
        df = self.engineer_features(df, is_training=False)

        # Preprocess features
        X = self.preprocess_data(df, is_training=False)

        # Scale features
        X_scaled = self.scaler.transform(X)

        # Make prediction
        prediction = self.model.predict(X_scaled)[0]

        return prediction

    def save(self, directory='models'):
        """Save the model and artifacts"""
        try:
            # Create directory if it doesn't exist
            os.makedirs(directory, exist_ok=True)

            # Save model using pickle
            model_path = os.path.join(directory, 'model.pkl')
            with open(model_path, 'wb') as f:
                pickle.dump(self.model, f)
            print(f"Model saved to {model_path}")

            # Save scaler using pickle
            scaler_path = os.path.join(directory, 'scaler.pkl')
            with open(scaler_path, 'wb') as f:
                pickle.dump(self.scaler, f)
            print(f"Scaler saved to {scaler_path}")

            # Save feature info with feature names and category mappings
            feature_info = {
                'feature_columns': self.feature_names,
                'category_mappings': self.category_mappings
            }

            # Print debug info
            print("\nFeature info to be saved:")
            print(f"Number of feature columns: {len(self.feature_names)}")
            print(f"Number of categories: {len(self.category_mappings)}")

            # Save to temporary file first
            feature_info_path = os.path.join(directory, 'feature_info.json')
            temp_path = feature_info_path + '.tmp'
            with open(temp_path, 'w') as f:
                json.dump(feature_info, f, indent=2)

            # Verify the temporary file was written correctly
            with open(temp_path, 'r') as f:
                saved_info = json.load(f)
                if not saved_info or 'feature_columns' not in saved_info:
                    raise ValueError("Failed to save feature info correctly")

            # Move temporary file to final location
            os.replace(temp_path, feature_info_path)
            print(f"Feature info saved to {feature_info_path}")

            print(f"\nAll artifacts saved to {directory}/")
        except Exception as e:
            print(f"Error saving artifacts: {str(e)}")
            raise

    def load(self, directory='models'):
        """Load the model and artifacts"""
        # Load model
        self.model = lgb.Booster(model_file=os.path.join(directory, 'model.txt'))

        # Load scaler
        self.scaler = joblib.load(os.path.join(directory, 'scaler.joblib'))

        # Load feature info
        with open(os.path.join(directory, 'feature_info.json'), 'r') as f:
            self.feature_info = json.load(f)

        print("Model and artifacts loaded successfully!")


In [10]:
def load_and_explore_data():
    """Load and clean the dataset"""
    # Load data with a reasonable limit
    df = pd.read_csv('../data/vehicles.csv', nrows=200000)

    # Keep only relevant columns
    columns_to_keep = ['year', 'manufacturer', 'model', 'condition', 'odometer',
                      'fuel', 'title_status', 'transmission', 'drive', 'type',
                      'paint_color', 'price']
    df = df[columns_to_keep]

    # Basic cleaning
    df = df.dropna(subset=['price'])  # Drop rows with missing prices
    df = df[(df['price'] >= 100) & (df['price'] <= 100000)]  # Filter price range

    # Convert categorical columns to lowercase
    categorical_cols = ['manufacturer', 'model', 'condition', 'fuel', 'title_status',
                       'transmission', 'drive', 'type', 'paint_color']
    for col in categorical_cols:
        if col in df.columns:
            df[col] = df[col].str.lower()

    # Fill missing values
    df['odometer'] = df['odometer'].fillna(df['odometer'].mean())
    df['year'] = df['year'].fillna(df['year'].median())
    for col in categorical_cols:
        if col in df.columns:
            df[col] = df[col].fillna('unknown')

    return df


In [11]:
def prepare_features(data, is_training=True):
    """
    Prepare features for model training or prediction.
    Args:
        data: DataFrame for training or dict/Series for prediction
        is_training: bool, whether preparing for training or prediction
    """
    if is_training:
        df = data.copy()
        # Drop rows with missing prices for training data
        df = df.dropna(subset=['price'])

        # Fill missing values
        df['odometer'] = df['odometer'].fillna(df['odometer'].mean())
        df['year'] = df['year'].fillna(df['year'].median())

        numeric_features = ['year', 'odometer']
        categorical_features = ['manufacturer', 'model', 'condition', 'fuel',
                              'title_status', 'transmission', 'drive', 'type',
                              'paint_color']

        # Convert categorical columns to lowercase and fill missing values
        for col in categorical_features:
            df[col] = df[col].str.lower()
            df[col] = df[col].fillna('unknown')

        # Calculate derived features
        current_year = datetime.now().year
        df['age'] = current_year - df['year']
        df['miles_per_year'] = df['odometer'] / df['age'].clip(lower=0.1)
        df['age_mileage'] = df['age'] * df['odometer']

        numeric_features.extend(['age', 'miles_per_year', 'age_mileage'])

        # Create dummy variables for categorical features
        dummies = pd.get_dummies(df[categorical_features], prefix=categorical_features)

        # Store category mapping
        category_mapping = {}
        for col in categorical_features:
            category_mapping[col] = df[col].unique().tolist()

        # Combine features
        features = pd.concat([df[numeric_features], dummies], axis=1)

        # Store feature information
        feature_info = {
            'numeric_features': numeric_features,
            'categorical_features': categorical_features,
            'category_mapping': category_mapping,
            'all_features': features.columns.tolist()
        }

        return features, df['price'], feature_info

    else:
        # Handle single vehicle prediction
        if isinstance(data, dict):
            df = pd.DataFrame([data])
        else:
            df = pd.DataFrame([data.to_dict()])

        # Load feature info
        with open('../models/feature_info.json', 'r') as f:
            feature_info = json.load(f)

        numeric_features = feature_info['numeric_features']
        categorical_features = feature_info['categorical_features']
        category_mapping = feature_info['category_mapping']
        all_features = feature_info['all_features']

        # Convert categorical columns to lowercase
        for col in categorical_features:
            df[col] = df[col].str.lower()
            df[col] = df[col].fillna('unknown')

        # Calculate derived features
        current_year = datetime.now().year
        df['age'] = current_year - df['year']
        df['miles_per_year'] = df['odometer'] / df['age'].clip(lower=0.1)
        df['age_mileage'] = df['age'] * df['odometer']

        # Create dummy variables
        dummies = pd.get_dummies(df[categorical_features], prefix=categorical_features)

        # Ensure all training features are present
        for feature in all_features:
            if feature not in dummies.columns and feature not in numeric_features:
                dummies[feature] = 0

        # Combine features
        features = pd.concat([df[numeric_features], dummies[all_features[len(numeric_features):]]], axis=1)

        return features[all_features]


In [12]:
def train_model(X_train, y_train, X_val, y_val):
    """Train a LightGBM model with early stopping."""
    train_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

    params = {
        'objective': 'regression',
        'metric': 'l1',
        'boosting_type': 'gbdt',
        'verbose': -1
    }

    callbacks = [
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(period=100)
    ]

    model = lgb.train(
        params,
        train_data,
        num_boost_round=1000,
        valid_sets=[val_data],
        callbacks=callbacks
    )

    return model


In [13]:
def save_model(model, scaler, feature_info):
    """Save the model, scaler, and feature information."""
    os.makedirs('../models', exist_ok=True)

    # Save model
    model.save_model('../models/model.txt')

    # Save scaler
    with open('../models/scaler.pkl', 'wb') as f:
        pickle.dump(scaler, f)

    # Save feature information
    with open('../models/feature_info.json', 'w') as f:
        json.dump(feature_info, f)


In [ ]:

# def main():
"""Main function to train and save the model."""
# print("Loading and preparing data...")
df = load_and_explore_data()

print("Engineering features...")
X, y, feature_info = prepare_features(df, is_training=True)

print("Splitting data...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Scaling features...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Training model...")
model = train_model(X_train_scaled, y_train, X_test_scaled, y_test)

print("Saving model and artifacts...")
save_model(model, scaler, feature_info)

# Test prediction
test_vehicle = {
    'year': 2014,
    'manufacturer': 'BMW',
    'model': '320i',
    'condition': 'excellent',
    'odometer': 112112,
    'fuel': 'gas',
    'title_status': 'clean',
    'transmission': 'automatic',
    'drive': 'rwd',
    'type': 'sedan',
    'paint_color': 'white'
}

X_pred = prepare_features(test_vehicle, is_training=False)
X_pred_scaled = scaler.transform(X_pred)
prediction = model.predict(X_pred_scaled)[0]

print(f"\nPredicted price for test vehicle: ${prediction:,.2f}")

# if __name__ == "__main__":
#     main()

Engineering features...
Splitting data...
Scaling features...
